install lib

In [ ]:
!pip install -r requirements.txt 

import all lib in project

In [1]:
import requests
import gspread
import os
from google.oauth2 import service_account
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from datetime import datetime

use creds for BQ/google_sheet
and url exchange rate


In [3]:
path_gcp_conn_id = 'PATH_YOUR_CREDS.json' 
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

gcp_conn_id = service_account.Credentials.from_service_account_file(path_gcp_conn_id)

url_exchange_rate = 'https://bank.gov.ua/NBU_Exchange/exchange_site?start=20240506&end=20240512&valcode=eur&sort=exchangedate&order=desc&json'


# https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={datetime.now()}&json ссылка на дату за один день
url_exchange_rate_today = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={datetime.today()}&json"

In [4]:
response = requests.get(url_exchange_rate)
df_exchange_rate = pd.DataFrame(response.json())
df_exchange_rate

,exchangedate,r030,cc,txt,enname,rate,units,rate_per_unit,group,calcdate
0,12.05.2024,978,EUR,Євро,Euro,42.4312,1,42.4312,1,09.05.2024
1,11.05.2024,978,EUR,Євро,Euro,42.4312,1,42.4312,1,09.05.2024
2,10.05.2024,978,EUR,Євро,Euro,42.4312,1,42.4312,1,09.05.2024
3,09.05.2024,978,EUR,Євро,Euro,42.3168,1,42.3168,1,08.05.2024
4,08.05.2024,978,EUR,Євро,Euro,42.3087,1,42.3087,1,07.05.2024
5,07.05.2024,978,EUR,Євро,Euro,42.3983,1,42.3983,1,06.05.2024
6,06.05.2024,978,EUR,Євро,Euro,42.3359,1,42.3359,1,03.05.2024


download exchange rate with date

In [5]:
project_id = 'applied-honor-354909'
table_id = 'vector_vs.exchange_rate'

df_exchange_rate.to_gbq(
    table_id, 
    project_id, 
    if_exists="append", 
    credentials=gcp_conn_id,
)    

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 10485.76it/s]


In [28]:
# correct columns name
def remove_brackets(column_name):
    return column_name.replace('(', '').replace(')', '')

# correct type float64
def convert_format(df, columns_check_ids):
    for column_id in columns_check_ids:
        df.iloc[:, column_id] = df.iloc[:, column_id].str.replace(',', '.')
        for index, value in df.iloc[:, column_id].items():
            try:
                float_value = float(value)
                int_value = int(float_value)
                df.iloc[index, column_id] = int_value
            except ValueError:
                pass
    return df.astype(str)
          
# function for connect google sheet, and download sheet in BQ
def download_excel_to_gbq(sheet_id: str, scope: str, project_id, gcp_conn_id, columns_check_ids, distinct):
    client = gspread.authorize(gcp_conn_id.with_scopes(scope))
    sheet_place = client.open_by_key(sheet_id)
    dfs = []

    for worksheet in sheet_place.worksheets():
        data_sheet = worksheet.get_all_values()
        table_id=f"vector_vs.{worksheet.id}{distinct}"
        df_sheet = pd.DataFrame(data_sheet[1:], columns=[remove_brackets(col) for col in data_sheet[0]])
        # df_sheet = convert_format(df_sheet, columns_check_ids)

        df_sheet.to_gbq(
            table_id,
            project_id,
            if_exists='append',
            credentials=gcp_conn_id
        )
        dfs.append(df_sheet)
    return dfs

In [ ]:
client = gspread.authorize(gcp_conn_id.with_scopes(scope))
sheet_place = client.open_by_key(sheet_id)
data_sheet = worksheet.get_all_values()

In [19]:
# fixme; add list in float sience number
# fixme: list sheet_id
# fixme: translate columns name 

In [24]:
project_id = 'applied-honor-354909'
table_id = 'vector_vs.exchange_rate'
distinct = 'test_table'
sheet_id = 'YOUR_SHEED_ID'
distinct = 'test'

columns_check_ids = [0]
catalog = download_excel_to_gbq(sheet_id, scope, project_id, gcp_conn_id, columns_check_ids, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 6413.31it/s]


[        Номенклатура_value         Опис
 0      1290110000000000000      Товар 1
 1      1289180000000000000      Товар 2
 2      1289180000000000000      Товар 3
 3      1287750000000000000      Товар 4
 4      1288450000000000000      Товар 5
 ...                    ...          ...
 16815  1292030000000000000  Товар 16816
 16816  1290920000000000000  Товар 16817
 16817  1291090000000000000  Товар 16818
 16818  1291090000000000000  Товар 16819
 16819  1291780000000000000  Товар 16820
 
 [16820 rows x 2 columns],
             КонтрагентCod Опис контрагента
 0     1288710000000000000     Контрагент 1
 1     1287780000000000000     Контрагент 2
 2     1288370000000000000     Контрагент 3
 3     1288830000000000000     Контрагент 4
 4     1289360000000000000     Контрагент 5
 ...                   ...              ...
 5764  1289290000000000000                 
 5765  4640900000000000000                 
 5766  1291840000000000000                 
 5767  1291910000000000000             

Как видим большая проблема в том что оно не хочет переоброзовывать правильно в числовой формат, по этому отдельно в google sheet ставлю как число, а потом как строку чтобы считало нормально и загрузило, остальное можно будет поправить в BQ


Так же в следующем примере показываем что нужно поменять название колонок на латыницу, а то данные вовсе не отображаються в DWH

In [25]:
project_id = 'applied-honor-354909'
distinct = 'test_table'
sheet_id = 'YOUR_SHEED_ID'
distinct = 'test_rename_columns'

columns_check_ids = [0]
catalog = download_excel_to_gbq(sheet_id, scope, project_id, gcp_conn_id, columns_check_ids, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 6403.52it/s]


[             catalog_value  description
 0      1290000000000000000      Товар 1
 1      1290000000000000000      Товар 2
 2      1290000000000000000      Товар 3
 3      1290000000000000000      Товар 4
 4      1290000000000000000      Товар 5
 ...                    ...          ...
 16815  1290000000000000000  Товар 16816
 16816  1290000000000000000  Товар 16817
 16817  1290000000000000000  Товар 16818
 16818  1290000000000000000  Товар 16819
 16819  1290000000000000000  Товар 16820
 
 [16820 rows x 2 columns],
              counterparty   description
 0     1290000000000000000  Контрагент 1
 1     1290000000000000000  Контрагент 2
 2     1290000000000000000  Контрагент 3
 3     1290000000000000000  Контрагент 4
 4     1290000000000000000  Контрагент 5
 ...                   ...           ...
 5764  1290000000000000000              
 5765  4640000000000000000              
 5766  1290000000000000000              
 5767  1290000000000000000              
 5768  1290000000000000000  

In [26]:
project_id = 'applied-honor-354909'
distinct = 'test_table'
sheet_id = 'YOUR_SHEED_ID'
distinct = ''

columns_check_ids = [0]
catalog = download_excel_to_gbq(sheet_id, scope, project_id, gcp_conn_id, columns_check_ids, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 7121.06it/s]


[                      catalog_value  description
 0      1 290 107 951 398 930 000.00      Товар 1
 1      1 289 175 564 849 130 000.00      Товар 2
 2      1 289 176 392 011 300 000.00      Товар 3
 3      1 287 746 568 254 190 000.00      Товар 4
 4      1 288 451 561 004 780 000.00      Товар 5
 ...                             ...          ...
 16815  1 292 029 263 606 390 000.00  Товар 16816
 16816  1 290 922 650 120 970 000.00  Товар 16817
 16817  1 291 092 287 805 410 000.00  Товар 16818
 16818  1 291 092 293 867 170 000.00  Товар 16819
 16819  1 291 776 132 891 810 000.00  Товар 16820
 
 [16820 rows x 2 columns],
                       counterparty   description
 0     1 288 709 057 205 030 000.00  Контрагент 1
 1     1 287 777 636 995 400 000.00  Контрагент 2
 2     1 288 365 007 388 790 000.00  Контрагент 3
 3     1 288 829 993 090 490 000.00  Контрагент 4
 4     1 289 358 637 481 140 000.00  Контрагент 5
 ...                            ...           ...
 5764  1 289 288 874 

загружаем данные которые уже максимально нам подходят

функция для загруки файла, уже приведенного к правильному формату с колонками в дальнейшом при необходимости названия можно поменять уже в BQ

In [46]:
def excel_to_gbq(file_path, project_id, gcp_conn_id, distinct):
    
    xl = pd.ExcelFile(file_path)
    dfs = {}
    
    for sheet_name in xl.sheet_names:
        dfs[sheet_name] = xl.parse(sheet_name)
        table_id=f"vector_vs.{distinct}_{sheet_name}"
        pandas_gbq.to_gbq(dfs[sheet_name].astype(str),
            table_id,
            project_id,
            if_exists='append',
            credentials=gcp_conn_id
        )
    return dfs

In [47]:
project_id = 'applied-honor-354909'
distinct = 'order'
file_path = 'FILE_PATH.xlsx'

catalog = excel_to_gbq(file_path, project_id, gcp_conn_id, distinct)
catalog

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 890.32it/s]


{'2018':                   Recorder_type                Recorder_value  row_n  Active  \
 0      Document_ЗаказПокупателя  1 290 507 094 376 650 000,00      1    True   
 1      Document_ЗаказПокупателя  1 290 507 095 628 210 000,00      1    True   
 2      Document_ЗаказПокупателя  1 290 518 991 834 640 000,00      1    True   
 3      Document_ЗаказПокупателя  1 290 440 574 153 040 000,00      1    True   
 4      Document_ЗаказПокупателя  1 290 281 466 605 450 000,00      1    True   
 ...                         ...                           ...    ...     ...   
 52170  Document_ЗаказПокупателя  1 290 554 445 900 500 000,00      6    True   
 52171  Document_ЗаказПокупателя  1 290 554 445 900 500 000,00      8    True   
 52172  Document_ЗаказПокупателя  1 290 558 950 284 700 000,00      1    True   
 52173  Document_ЗаказПокупателя  1 290 558 950 284 700 000,00      2    True   
 52174  Document_ЗаказПокупателя  1 290 560 494 934 660 000,00      1    True   
 
        RecordType

все подходящие под формат, загрузка прошла успешно

In [48]:
project_id = 'applied-honor-354909'
distinct = 'order'
file_path = 'FILE_PATH.xlsx'

catalog = excel_to_gbq(file_path, project_id, gcp_conn_id, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 8924.05it/s]


{'2021':                   Recorder_type                Recorder_value  row_n  Active  \
 0      Document_ЗаказПокупателя  1 291 212 858 509 930 000,00      1    True   
 1      Document_ЗаказПокупателя  1 291 218 067 336 230 000,00      1    True   
 2      Document_ЗаказПокупателя  1 291 221 697 056 680 000,00      1    True   
 3      Document_ЗаказПокупателя  1 291 232 732 633 390 000,00      1    True   
 4      Document_ЗаказПокупателя  1 291 233 539 293 560 000,00      1    True   
 ...                         ...                           ...    ...     ...   
 83265  Document_ЗаказПокупателя  1 291 523 115 575 340 000,00      2    True   
 83266  Document_ЗаказПокупателя  1 291 523 115 575 340 000,00      4    True   
 83267  Document_ЗаказПокупателя  1 291 523 115 575 340 000,00      5    True   
 83268  Document_ЗаказПокупателя  1 291 523 115 575 340 000,00      7    True   
 83269  Document_ЗаказПокупателя  1 291 523 115 575 340 000,00      9    True   
 
        RecordType

все подходящие под формат, загрузка прошла успешно

In [49]:
project_id = 'applied-honor-354909'
distinct = 'order'
file_path = 'FILE_PATH.xlsx'

catalog = excel_to_gbq(file_path, project_id, gcp_conn_id, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 7133.17it/s]


{'2023':                   Recorder_type                Recorder_value  row_n  Active  \
 0      Document_ЗаказПокупателя  1 291 913 460 781 710 000,00      1    True   
 1      Document_ЗаказПокупателя  1 291 914 307 053 940 000,00      1    True   
 2      Document_ЗаказПокупателя  1 291 914 337 960 130 000,00      1    True   
 3      Document_ЗаказПокупателя  1 291 916 895 594 770 000,00      1    True   
 4      Document_ЗаказПокупателя  1 291 916 897 639 880 000,00      1    True   
 ...                         ...                           ...    ...     ...   
 98932  Document_ЗаказПокупателя  1 291 902 460 586 900 000,00      1    True   
 98933  Document_ЗаказПокупателя  1 291 902 460 586 900 000,00      3    True   
 98934  Document_ЗаказПокупателя  1 291 903 203 709 850 000,00      1    True   
 98935  Document_ЗаказПокупателя  1 291 903 203 709 850 000,00      2    True   
 98936  Document_ЗаказПокупателя  1 291 904 806 764 080 000,00      1    True   
 
        RecordType

In [49]:
project_id = 'applied-honor-354909'
distinct = 'order'
file_path = 'FILE_PATH.xlsx'

catalog = excel_to_gbq(file_path, project_id, gcp_conn_id, distinct)
catalog

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 7133.17it/s]


{'2023':                   Recorder_type                Recorder_value  row_n  Active  \
 0      Document_ЗаказПокупателя  1 291 913 460 781 710 000,00      1    True   
 1      Document_ЗаказПокупателя  1 291 914 307 053 940 000,00      1    True   
 2      Document_ЗаказПокупателя  1 291 914 337 960 130 000,00      1    True   
 3      Document_ЗаказПокупателя  1 291 916 895 594 770 000,00      1    True   
 4      Document_ЗаказПокупателя  1 291 916 897 639 880 000,00      1    True   
 ...                         ...                           ...    ...     ...   
 98932  Document_ЗаказПокупателя  1 291 902 460 586 900 000,00      1    True   
 98933  Document_ЗаказПокупателя  1 291 902 460 586 900 000,00      3    True   
 98934  Document_ЗаказПокупателя  1 291 903 203 709 850 000,00      1    True   
 98935  Document_ЗаказПокупателя  1 291 903 203 709 850 000,00      2    True   
 98936  Document_ЗаказПокупателя  1 291 904 806 764 080 000,00      1    True   
 
        RecordType

Выводы:

1. чтобы все загружать с гугл диска необходимо название колонок латиницей # такую же проблему заметил и с использованием AWS
2. формат приведения не должен быть везде общим, можно так же хранить название столбцов и к ним обращатся при загрузке(чтобы не менять названия в основных таблицах)
3. если числа большого формата лучше передавать их строкой, потом все меняется
4. но лучше всего хэшировать информацию, так более надежденее сравнивать в дальнешом(в основном выделяется 64бита)
5. для запуска такого скрипта из того что я использовал можно использовать airflow, так же можно попробовать настроить такое через dbt слышал там есть уже что-то подобное
6. для больших файлов лучше их разделять не всегда есть возможность их загружать, в ином случае производить выгрузку с сервера какого-то
7. ниже описанно то как переводил стобцы в таблицах


Чтобы проверить:

1. отослать мне почту, добавлю в BigQuery
2. там есть vector_vs куда добавлял все таблицы
3. base_models таблицы где переименование и смена типов данных
4. staging_model где все stg модели которые уникальные данные через DISTINCT/если есть по чем сделать дедупликацию то сделаю
5. marts_or_reports там лежит mart__orders который необходим для построения аналитики его вывод привиден ниже сам запрос, а так же то что после него получаем

Если необходимо проверить скрипт:
1. написать мне отошлю json.key, а так же дам доступ до google sheet с которого необходимо взять айди или же выгрузить файлы


то что находится в марте окончательно, а так же еще загрузка данных можно уменьшить количество данных выбрать только нужные стобцы
так же сам запрос для этого марта

In [54]:
orders = pd.DataFrame()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="PATH_YOUR_CREDS.json"
client = bigquery.Client()

query = f"""
SELECT * FROM `applied-honor-354909.marts_or_reports.mart__orders`
"""
query_job = client.query(query)
df = query_job.to_dataframe()
df.to_csv('order.csv', index=False)
df

,recorder_type,recorder_value,row_n,is_active,record_type,agreement_counterparty_value,order_client_linkflag,order_client_type,order_client_value,status_part_value,...,issued_the_invoice_value,order_id,order_status,order_created_at,counterparty_description,catalog_description,rate_for_one_unit,цена в долларах,сумма упр в доллорах,сумма упр зпдв в долларах
0,Document_ЗаказПокупателя,1.290270e+20,1,True,1,1.288436e+20,8,Document_ЗаказПокупателя,1.290270e+20,4.629441e+20,...,1.289151e+20,ТВК00000186 от 15.01.2018,Відвантаження,10.01.2018,None,None,33.652111,66.84,43.44,52.13
1,Document_ЗаказПокупателя,1.290275e+20,1,True,1,1.288436e+20,8,Document_ЗаказПокупателя,1.290275e+20,4.629441e+20,...,1.289151e+20,ТВК00000344 от 15.01.2018,Відвантаження,15.01.2018,None,None,34.634916,111.12,72.24,86.68
2,Document_ЗаказПокупателя,1.290270e+20,5,True,1,1.288436e+20,8,Document_ЗаказПокупателя,1.290270e+20,4.629441e+20,...,1.289151e+20,ТВК00000186 от 15.01.2018,Відвантаження,10.01.2018,None,None,33.652111,112.13,72.89,87.46
3,Document_ЗаказПокупателя,1.290270e+20,6,True,1,1.288436e+20,8,Document_ЗаказПокупателя,1.290270e+20,4.629441e+20,...,1.289151e+20,ТВК00000186 от 15.01.2018,Відвантаження,10.01.2018,None,None,33.652111,3.77,2.44,2.94
4,Document_ЗаказПокупателя,1.290275e+20,4,True,1,1.288436e+20,8,Document_ЗаказПокупателя,1.290275e+20,4.629441e+20,...,1.289151e+20,ТВК00000344 от 15.01.2018,Відвантаження,15.01.2018,None,None,34.634916,3.71,2.40,2.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518592,Document_ЗаказПокупателя,1.292042e+20,1,True,1,1.291637e+20,8,Document_ЗаказПокупателя,1.292042e+20,4.629441e+20,...,1.288466e+20,ТВК00019191 от 22.08.2023,Підтверджено,22.08.2023,None,None,39.889000,17.57,20.38,24.46
518593,Document_ЗаказПокупателя,1.292141e+20,1,True,1,1.291842e+20,8,Document_ЗаказПокупателя,1.292141e+20,4.629441e+20,...,1.289551e+20,ТВК00027667 от 15.12.2023,Підтверджено,15.12.2023,None,None,40.413000,0.71,1068.22,1281.86
518594,Document_ЗаказПокупателя,1.292152e+20,1,True,1,1.291842e+20,8,Document_ЗаказПокупателя,1.292152e+20,4.629441e+20,...,1.289551e+20,ТВК00028457 от 28.12.2023,Підтверджено,28.12.2023,None,None,41.633400,104.16,1599.94,1919.94
518595,Document_ЗаказПокупателя,1.292119e+20,1,True,1,1.291844e+20,8,Document_ЗаказПокупателя,1.292119e+20,4.629441e+20,...,1.288236e+20,ТВК00025641 от 20.11.2023,Підтверджено,20.11.2023,None,None,39.258200,1293.93,5434.51,6521.40


То что находится внутри марта 

In [ ]:

WITH stg__orders AS (
  SELECT * FROM `staging_model.stg__orders`
)
, stg__exchange_rate AS (
  SELECT * FROM `staging_model.stg__exchange_rate`
)
, stg__catalog AS (
  SELECT * FROM `staging_model.stg__catalog` 
)
, stg__counterparty AS (
  SELECT * FROM `staging_model.stg__counterparty`
)
, amount_in_dollar AS (
  SELECT stg__orders.*
    , stg__counterparty.description AS counterparty_description
    , stg__catalog.description AS catalog_description
    , stg__exchange_rate.rate_for_one_unit
    , ROUND(amount / rate_for_one_unit, 2) AS `цена в долларах`
    , ROUND(control_amount / rate_for_one_unit, 2) AS `сумма упр в доллорах`
    , ROUND(amount_control_zpdv / rate_for_one_unit, 2) AS `сумма упр зпдв в долларах`
  FROM stg__orders
  LEFT JOIN stg__exchange_rate ON stg__orders.order_created_at = stg__exchange_rate.exchange_date_at
  LEFT JOIN stg__catalog USING(catalog_value)
  LEFT JOIN stg__counterparty USING(counterparty)
)
SELECT *
FROM amount_in_dollar
-- WHERE counterparty_description IS NOT NULL OR catalog_description IS NOT NULL


In [ ]:
Recorder_type
Recorder_value
row_n
Active
RecordType
ДоговорКонтрагента_value agreement_counterpatry_value
ЗаказПокупателя_linkflag order_client_linkflag
ЗаказПокупателя_type order_client_type
ЗаказПокупателя_value order_client_value
СтатусПартии_value status_part_value
Номенклатура_value catalog_value
ХарактеристикаНоменклатуры_value nomenclature_description_value
Цена amount
ЕдиницаИзмерения_value amount_exchange_rate_value
ПроцентСкидкиНаценки percentage_discounts_surcharges
ПроцентАвтоматическихСкидок percentage_discounts_automatic
УсловиеАвтоматическойСкидки_value automatic_discount_condition
ЗначениеУсловияАвтоматическойСкидки_linkflag meaning_of_automatic_discount_conditions 
ЗначениеУсловияАвтоматическойСкидки_num meaning_of_automatic_discount_conditions_num
ЗначениеУсловияАвтоматическойСкидки_type meaning_of_automatic_discount_conditions_type
ЗначениеУсловияАвтоматическойСкидки_value meaning_of_automatic_discount_conditions_value
СтавкаНДС_value VAT_rate_value
Количество order_quantity
СуммаВзаиморасчетов mutual_settlement_amount
СуммаУпрЗПДВ  amount_control_ZPDV
Контрагент(Cod) counterparty
РМ
Ставка bit_rate
СуммаУпр control_amount
ВалютаДокумента_value document_currency_value
КратностьВзаиморасчетов multiplicity_of_mutual_settlements
КурсВзаиморасчетов mutual_settlement_course
ТипЦен_value amount_type_value
Document_ЗаказПокупателя.ураКратностьДокумента	document_order_client_document_multiplicity
Document_ЗаказПокупателя.ураВалютаДокумента_value	document_order_client_currency
ИТЕК_ДатаСоздания ITEC_created_at
ПересчетПоКурсуВалюты	conversion_at_the_currency_rate
Категорія Продукту product_category
Дата первой покупки first_order_created_at
_СтатусЗаказа_value order_status_value
Ответственный_value responsible_value
КтоВыписалСчет_value issued_the_invoice_value
Заказ order
СтатусЗамовлення order_status